# Attention Mechanism

## Seq2Seq 모델의 문제점
+ 문제점1 : 하나의 **고정 길이 벡터**에 모든 정보를 압축 → 정보 손실 발생
+ 문제점2 : RNN의 문제점인 기울기 소실 문제 (Vanishing Gradient)가 여전히 발생
  + 기계번역 분야에서 입력 문장이 길면 번역 품질이 떨어지는 현상으로 나타남
* seq2seq 문제 개선을 위해 **Attention** Mechanism 고안
  - 입력 시퀀스가 길어지면 출력 시퀀스의 정확도가 떨어지는 것을 보정(개선)



# Attention 의 아이디어
* Attention Mechanism은 디코더가 출력단어를 예측하는 매 시점(time step)마다 인코더의 '전체 입력 문장'을 다시 한번 참조!
* 이때 전체 입력 문장을 단순히 (동일 비율로) 참조하는 것이 아니라, **예측할 단어와 연관이 있는 단어를 집중(Attention)**해서 참조하게 된다

# Attention Function
어텐션 함수

![](https://wikidocs.net/images/page/22893/%EC%BF%BC%EB%A6%AC.PNG)

- 어텐션을 함수로 표현하면 주로 다음과 같이 표현됩니다

  - **Attention(Q, K, V) = Attention Value**
  - Q: Query
  - K: Key
  - V: Value

- 어텐션 함수는 주어진 '쿼리(Query)'에 대해서 모든 '키(Key)'와의 **유사도**를 각각 구합니다.
- 그리고 구해낸 이 **유사도**를 키와 맵핑되어있는 각각의 **'값(Value)'에 반영**해줍니다.
- 그리고 유사도가 반영된 **'값(Value)' 들을 모두 더해서 리턴**합니다.
  - 이를 (편의상) **어텐션 값(Attention Value)**이라고 하겠습니다.

- 지금부터 배우게 되는 Seq2Seq + 어텐션 모델에서 Q, K, V에 해당되는 각각의 Query, Keys, Values는 각각 다음과 같습니다.

```
Q = Query : t 시점(타임스텝)의 디코더 셀에서의 은닉 상태
K = Keys : 모든 시점(타임스텝)의 인코더 셀의 은닉 상태들
V = Values : 모든 시점(타임스텝)의 인코더 셀의 은닉 상태들
```

# Attention Mechanism 과정
* Attention Mechanism 중 가장 기초적이고 수식적으로 이해하기 쉬운 **dot-product attention** 을 통해 배워보겠습니다


* Attention Mechanism 과정 (단계)
  1. attention score 계산
  1. 소프트맥스 함수를 통한 attention distribution 계산
  1. 각 인코더의 어텐션 가중치와 은닉 상태를 가중합하여 어텐션 값 계산
  1. 어텐션 값과 디코더의 t 시점의 은닉 상태를 연결
  1. 출력층 연산의 입력이 되는 $\tilde{s}_t$ 계산
  1. $\tilde{s}_t$ 를 출력층의 입력으로 사용합니다

[전체적인 개요(예시)]

![](https://wikidocs.net/images/page/22893/dotproductattention1_final.PNG)

- ↑ 위 그림은 디코더의 '세번째 LSTM 셀'에서 출력 단어를 '예측'할 때,
어텐션 메커니즘을 사용하는 모습을 보여줍니다.

- 디코더의 첫번째, 두번째 LSTM 셀은 이미 어텐션 메커니즘을 통해 je와 suis를 예측하는 과정을 거쳤다고 가정합니다.

- 디코더의 세번째 LSTM 셀은 출력 단어를 예측하기 위해서 **인코더의 모든 입력 단어들의 정보를 다시 한번 참고**하고자 합니다.

- 중간 과정에 대한 설명은 현재는 생략, 주목할 것은 인코더의 **소프트맥스 함수** !!.

- 소프트맥스 함수를 통해 나온 결과값은 I, am, a, student 단어 각각이 '출력 단어를 예측'할 때 '얼마나 도움'이 되는지의 정도를 수치화한 값.

- 빨간 직사각형의 크기로 소프트맥스 함수의 결과값의 크기를 표현. 직사각형의 크기가 클 수록 도움이 되는 정도의 크기가 큽니다.

- 각 입력 단어가 디코더의 예측에 도움이 되는 정도가 수치화하여 측정되면 이를 **'하나의 정보'**로 담아서 디코더로 전송됩니다. 그림에서는 초록색 삼각형이 이에 해당됩니다.

- 결과적으로, 디코더는 출력 단어를 더 정확하게 예측할 확률이 높아진다.

## 1) Attention Score 계산

![](https://wikidocs.net/images/page/22893/dotproductattention2_final.PNG)

*   인코더의 각 시점의 은닉 상태는 $h_n$ 라고 합시다

*   디코더의 현재 시점 (timestep) t의 디코더 은닉 상태를 $s_t$ 라고 합시다

*   위 예제에선 인코더의 은닉 상태와 디코더의 은닉상태의 차원이 같다고 가정합니다.  위 그림에선 인코더의 은닉 상태와 디코더의 은닉 상태가 동일하게 차원이 4입니다.

* 이전의 seq2seq 에서 배웠던 내용!
  - 현재 시점(timestep) t 에서 출력 단어를 예측하기 위해서 디코더의 셀은 두 개의 입력값을 필요로 했다
    - 이전 시점 t-1의 은닉 상태
    - 이전 시점 t-1에 나온 출력 단어.

* 그런데 어텐션 메커니즘에서는 출력 단어 예측에 또 다른 값을 필요하다.
  - 바로 **어텐션 값(Attention Value)**이라는 새로운 값입니다.
  - t번째 타임스텝의 단어를 예측하기 위한 어텐션 값을 $a_t$
이라고 정의하자.

* **어텐션 값** 이 현재 시점 t 에서의 출력 예측에 구체적으로 어떻게 반영되는지는 나중에 설명하고.

* 지금부터 배우는 모든 과정은 $a_t$ 를 구하기 위한 여정입니다. 그리고 그 여정의 첫 걸음은 바로 **어텐션 스코어(Attention Score)**를 구하는 일입니다.

* **어텐션 스코어**란 현재 디코더의 타임스텝 t에서 단어를 예측하기 위해, 인코더의 모든 은닉 상태 각각이 디코더의 현 시점의 은닉 상태 $s_t$ 와 얼마나 유사한지를 판단하는 스코어값입니다

*   여기서는 dot-product attention 에선 이렇게 계산한다
  - '$s_t$ 를 전치(transpose)' 하고(결과 $s_t^T$) '각 은닉상태 $h_n$'와 내적(dot product)를 수행합니다.  즉 모든 어텐션 스코어 값은 '스칼라' 입니다.
  - 예를 들어 $s_t$ 와 인코더의 $i$번째 은닉상태와 어텐션 스코어의 계산 방법은 아래와 같다.
  - ![](https://wikidocs.net/images/page/22893/i%EB%B2%88%EC%A7%B8%EC%96%B4%ED%85%90%EC%85%98%EC%8A%A4%EC%BD%94%EC%96%B4_final.PNG)
  - 이 '어텐션 스코어 함수'를 정의해보면 다음과 같다<br>
    $score(s_t, h_i) = s_t^Th_i$
  - $s_t$와 인코더의 모든 은닉 상태의 어텐션 스코어의 모음값을 $e^t$ 라고 정의하면 → attention scores = $e^t = [s_t^Th_1, s_t^Th_2, ... , s_t^Th_n]$



## 2) Attention Distribution 계산
소프트 맥스 함수를 통한 attention distribution 계산


![](https://wikidocs.net/images/page/22893/dotproductattention3_final.PNG)



*   $e^t = [s_t^Th_1, s_t^Th_2, ... , s_t^Th_n$] 에 softmax를 적용해 '각각의 확률값'들을 계산
  - 이 값들은 합하면 1이 되는 확률분포 다.
  - 이를 **어텐션 분포(Attention Distribution)** 라고 하고
  - 각각의 값은 **어텐션 가중치(Attention Weight)**  라고 한다
  - 예를 들어 소프트맥스 함수를 적용하여 얻은 출력값인 I, am, a, student의 어텐션 가중치를 각각 0.1, 0.4, 0.1, 0.4라고 합시다. 이들의 합은 1입니다. 위의 그림은 각 인코더의 은닉 상태에서의 어텐션 가중치의 크기를 직사각형의 크기를 통해 시각화하였습니다. 즉, 어텐션 가중치가 클수록 직사각형이 큽니다.


*   내적이 크다면 확률값이 높아질 것이고, 작다면 확률값이 낮아짐
*   결국 확률값을 구하는 것은 예측할 단어와 연관이 있는 단어를 찾는 것
*   디코더의 시점 t 의 가중치 모음값인 어텐션 분포를 $\alpha^t$이라고 하면, 다음과 같은 식이 성립함
$$ \alpha^t = softmax(e^t) $$



## 3) Attention Value 계산
각 인코더의 어텐션 가중치와 은닉상태를 가중합하여 계산.

![](https://wikidocs.net/images/page/22893/dotproductattention4_final.PNG)

* 이제 지금까지 준비해온 정보들을 하나로 합치는 단계다.
* 어텐션의 최종 결과값을 얻기 위해서 각 인코더의 은닉 상태와 어텐션 가중치들을 곱하고, 최종적으로 모두 더합니다.
  - 즉! 가중합(Weighted Sum) 을 진행합니다
*   어텐션의 최종 결과. 즉, 어텐션 함수의 출력값인 어텐션 값(Attention Value) **$a_t$**
에 대한 식을 보여줍니다.<br>
  $a_t = \sum^N_{i=1}{\alpha^t_ih_i}$

* 이러한 어텐션 값 $a_t$은 종종 인코더의 문맥을 포함하고 있다고하여, **컨텍스트 벡터(context vector)**라고도 불립니다.
  * 앞서 배운 가장 기본적인 seq2seq에서는 인코더의 마지막 은닉 상태를 컨텍스트 벡터라고 부르는 것과 대조됩니다.

## 4) Attention Value 와 디코더의 t 시점의 은닉 상태를 연결
=> Concatenate

![](https://wikidocs.net/images/page/22893/dotproductattention5_final_final.PNG)


*   어텐션 함수의 최종값인 어텐션 값 $a_t$를 구했다면
*  $a_t$ 와 $s_t$와 연결(concatenate)하여 '예측' 연산에 사용할 하나의 벡터 $v_t$를 만들어 낸다.
*   이 $v_t$는 기존과는 다르게 '인코더의 정보'를 가지고 있고
* $v_t$ 가 $\hat y$ 예측연산의 입력으로 사용하게되므로 좀 더 좋은 성능의 예측을 수행할 수 있게 된다.  
* 이것이 어텐션 메커니즘의 핵심이다!



## 5) 출력층 연산의 입력이 되는 $\tilde{s}_t$ 계산

![](https://wikidocs.net/images/page/22893/st.PNG)
* 다음은 논문에서의 내용입니다
* $v_t$ 를 바로 출력층으로 보내기 전에 신경망 연산을 한 번 더 추가하였습니다.
* 가중치 행렬과 곱한 후에 하이퍼볼릭탄젠트($tanh$) 함수를 지나도록 하여 출력층 연산을 위한 새로운 벡터인 $\tilde{s}_t$
를 얻습니다.
* 어텐션 메커니즘을 사용하지 않는 seq2seq에서는 출력층의 입력이 t시점의 은닉 상태인 $s_t$
였던 반면, 어텐션 메커니즘에서는 출력층의 입력이 $\tilde{s}_t$ 가 되는 셈입니다.

* 이를 식으로 표현하면 다음과 같습니다.
  - $\tilde{s}_t = tanh(W_c[a_t ; s_t] + b_c)$
    - $W_c$는 학습 가능한 가중치 행렬,
    - $b_c$는 bias(편향, 절편)  (위 그림에선 편향은 생략)


## 6) $\tilde{s}_t$ 를 출력층의 입력으로 사용합니다

$\tilde{s}_t$ 를 출력층의 입력으로 사용하여 예측 벡터를 얻습니다.

$\hat{y}_t = Softmax(W_y\tilde{s}_t + b_y)$


# Attention Mechanism 종류
다양한 종류의 어텐션
* attention mechanism에는 어텐션 스코어 계산 방식(어센텬 스코어 함수)의 차이에 따라 다양한 종류가 존재


이 름                 | 스 코 어&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|Defined By
----------------------|-----------------------|--
dot|$score(s_t, h_i) = s_t^Th_i$|Luong et al. (2015)
scaled dot|$score(s_t, h_i) = \frac{s_t^Th_i}{\sqrt n}$|Vaswani et al. (2017)
general|$score(s_t, h_i) = s_t^TW_ah_i$|Luong et al. (2015)
concat|$score(s_t, h_i) = W_a^Ttanh(W_b[s_t;h_i])$|Bahdanau et al. (2015)
location - base|$a_t = softmax(W_as_t)$|Luong et al. (2015)
  + $s_t$ : querys(t 시점에서의 디코더 셀의 은닉 상태)
  + $h_i$ : keys(모든 시점의 인코더 셀 은닉 상태)
  + $W_a, W_b$ : 학습 가능한 가중치 행렬

